# PyMongo

pymongo是使用python操作mongodb的推荐方式
## 安装PyMongo
conda install -y pymongo

## 使用前先开启mongod服务

```mongod --dbpath /home/alley/work/Dong/mongo/ --logpath /home/alley/work/Dong/mongo/log```
默认了host和port

## 连接数据库

In [5]:
import pymongo  
# client = pymongo.MongoClient()  # 默认了host和port
# client = pymongo.MongoClient('127.0.0.1', 27017)
client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
client

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)

## 获取一个数据库、集合

In [6]:
# 不存在的话会隐式创建，直到有数据写入时会成功显式创建
db = client.test  # 数据库
# db = client['test']
collection = db.test_collection  # 集合

## 文档（documents）

In [74]:
# 文档以bson格式存储，为二进制的json格式，类似于python中的字典，如下例子
import datetime
post = {
    "author": "Mike",
    "text": "My first blog post!",
    "tags": ["mongodb", "python", "pymongo"],
    "date": datetime.datetime.utcnow()
} 
post  # 文档中可包含python的数据类型，会自动转换成合适的格式

{'author': 'Mike',
 'text': 'My first blog post!',
 'tags': ['mongodb', 'python', 'pymongo'],
 'date': datetime.datetime(2020, 11, 16, 14, 53, 19, 697999)}

## 插入文档

In [33]:
# 插入一个文档
posts = db.posts  # 隐式创建一个posts的集合
# 插入一条文档,会自动创建一个“_id”的key，该值唯一，可手动添加该key
post_id = posts.insert_one(post).inserted_id  # 插入的同时获取_id
post_id

ObjectId('5fb27896afc3fb34723c3858')

In [34]:
db.list_collection_names()  # 打印所有集合名字

['posts']

## 单一查询

In [51]:
import pprint
pprint.pprint(posts.find_one())  # 查询第一个

{'_id': ObjectId('5fb27896afc3fb34723c3858'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 11, 16, 13, 3, 16, 833000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [36]:
pprint.pprint(posts.find_one({"author":"Mike"}))  # 查询符合条件的第一个

{'_id': ObjectId('5fb27896afc3fb34723c3858'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 11, 16, 13, 3, 16, 833000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [37]:
pprint.pprint(posts.find_one({"_id":post_id}))  # 通过_id查询

{'_id': ObjectId('5fb27896afc3fb34723c3858'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 11, 16, 13, 3, 16, 833000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [39]:
post_id_as_str = str(post_id)
post_id_as_str

'5fb27896afc3fb34723c3858'

In [40]:
posts.find_one({"_id": post_id_as_str}) # 查不到，转成str后与之前的不同

In [43]:
from bson.objectid import ObjectId  # 通过下面的例子可以将str转回来
pprint.pprint(posts.find_one({"_id": ObjectId(post_id_as_str)}))

{'_id': ObjectId('5fb27896afc3fb34723c3858'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 11, 16, 13, 3, 16, 833000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


## 批量插入

In [45]:
# python可以用单引号，但mongo shell里不行
# 注意前后插入的文档并没有完全相同的key，mongo没有模式限制
new_posts = [
    {
    'author': 'Mike',
    'text': 'Another post!',
    'tags': ['bulk', 'insert'],
    'date': datetime.datetime(2020, 11, 16, 21, 39)
    },
    {
    'author': 'Eliot',
    'title': 'MongoDB is fun',
    'text': 'easy too',
    'date': datetime.datetime(2020, 11, 17, 21, 39)
    }
]
result = posts.insert_many(new_posts)
result.inserted_ids  # 注意加了s

[ObjectId('5fb2819bafc3fb34723c3859'), ObjectId('5fb2819bafc3fb34723c385a')]

## 批量查询

In [83]:
cursor = db.posts.find()
cursor  # 返回一个cursor（游标可迭代）

In [84]:
for post in cursor:
    pprint.pprint(post)  # 第二次使用游标将无结果

{'_id': ObjectId('5fb27896afc3fb34723c3858'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 11, 16, 13, 3, 16, 833000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
{'_id': ObjectId('5fb2819bafc3fb34723c3859'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 11, 16, 21, 39),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}
{'_id': ObjectId('5fb2819bafc3fb34723c385a'),
 'author': 'Eliot',
 'date': datetime.datetime(2020, 11, 17, 21, 39),
 'text': 'easy too',
 'title': 'MongoDB is fun'}


## 计数

In [85]:
db.posts.count_documents({})  # {}不能省略

3

In [86]:
posts.count_documents({'author':'Mike'})

2

## 范围查询
MongoDB支持很多[高级查询](http://www.mongodb.org/display/DOCS/Advanced+Queries)

In [80]:
d = datetime.datetime(2020, 11, 17, 0)
# 查询日期小于d的同时按‘text’字段排序
for post in posts.find({"date":{'$lt': d}}).sort("text"):
    pprint.pprint(post)

{'_id': ObjectId('5fb2819bafc3fb34723c3859'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 11, 16, 21, 39),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}
{'_id': ObjectId('5fb27896afc3fb34723c3858'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 11, 16, 13, 3, 16, 833000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


## [索引（indexing）](https://docs.mongodb.com/manual/indexes/)
索引不仅可以加快查询速度，同时可以额外增加查询和存储文档的功能。
创建单一索引后，与该索引相同的文档将被拒绝插入

In [87]:
result = db.profiles.create_index([('user_id', pymongo.ASCENDING)], unique=True)
sorted(list(db.profiles.index_information()))  # 查询索引信息

['_id_', 'user_id_1']

In [88]:
user_profiles = [
    {
        'user_id': 211, 'name': 'Luke',
    },
    {
        'user_id': 212, 'name': 'Ziltoid'
    }
]
result = db.profiles.insert_many(user_profiles)

In [90]:
new_profile = {'user_id': 213, 'name': 'Drew'}
duplicate_profile = {'user_id': 212, 'name': 'Tommy'}
result = db.profiles.insert_one(new_profile)
result

result = db.profiles.insert_one(duplicate_profile) 
会报错，因为user_id相同了